In [2]:
import os
import sys
import numpy as np
import pandas as pd
from ast import literal_eval
pd.set_option('display.max_rows', None)

current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

from swp.utils.setup import seed_everything, set_device
from swp.datasets.phonemes import get_phoneme_to_id
from swp.utils.datasets import enrich_for_plotting, classify_error_positions

seed_everything()
device = set_device()
phoneme_to_id = get_phoneme_to_id()


Using MPS device


In [35]:
rlch = pd.read_csv("./evaluation/rlch.csv").sample(100)
rlcl = pd.read_csv("./evaluation/rlcl.csv").sample(100)
rlsh = pd.read_csv("./evaluation/rlsh.csv").sample(100)
rlsl = pd.read_csv("./evaluation/rlsl.csv").sample(100)
rsch = pd.read_csv("./evaluation/rsch.csv").sample(100)
rscl = pd.read_csv("./evaluation/rscl.csv").sample(100)
rssh = pd.read_csv("./evaluation/rssh.csv").sample(100)
rssl = pd.read_csv("./evaluation/rssl.csv").sample(100)

plc = pd.read_csv("./evaluation/plc.csv").sample(100)
pls = pd.read_csv("./evaluation/pls.csv").sample(100)
psc = pd.read_csv("./evaluation/psc.csv").sample(100)
pss = pd.read_csv("./evaluation/pss.csv").sample(100)

# concatenate all the datasets
test_equalized = pd.concat([rlch, rlcl, rlsh, rlsl, rsch, rscl, rssh, rssl, plc, pls, psc, pss], ignore_index=True)

order = [
    "Word", "Condition", "Lexicality", "Size", "Morphology",
    "Frequency", "Length", "Zipf Frequency", "Phonemes",
    "No Stress", "Part of Speech", "Root", "Lemma", "Zipf Lemma Frequency"
]

test_equalized = test_equalized[order]
print(test_equalized["Condition"].value_counts())

Condition
RLCH    100
RLCL    100
RLSH    100
RLSL    100
RSCH    100
RSCL    100
RSSH    100
RSSL    100
PLC     100
PLS     100
PSC     100
PSS     100
Name: count, dtype: int64
Lexicality
real      800
pseudo    400
Name: count, dtype: int64
Size
long     600
short    600
Name: count, dtype: int64
Morphology
complex    600
simple     600
Name: count, dtype: int64
Frequency
high    400
low     400
Name: count, dtype: int64
